In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [ ]:
data = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/train.csv")

In [ ]:
comments=data[0:20000]
comments

In [ ]:
#pd.DataFrame([x for x in np.where(df ==1, df.columns,'').flatten().tolist() if len(x) >0],columns= (["tags"]) )

In [ ]:
stopword = nltk.corpus.stopwords.words('english')
wn=nltk.WordNetLemmatizer()
st=nltk.PorterStemmer()

In [ ]:
def clean(text):
    text=re.sub(r'[^\w\s]',' ',text).lower()   ### Remove punctuation by removing non-space alpha-num characters
    text=re.sub(r'[\d]',' ',text).strip()     ##Remove numbers
#     # Clean some punctutations
#     text = re.sub('\n', ' \n ',text)
#     text = re.sub(r'([a-zA-Z]+)([/!?.])([a-zA-Z]+)',r'\1 \2 \3',text)
#     # Replace repeating characters more than 3 times to length of 3
#     text = re.sub(r'([*!?\'])\1\1{2,}',r'\1\1\1',text)    
#     # Add space around repeating characters
#     text = re.sub(r'([*!?\']+)',r' \1 ',text)    
#     # patterns with repeating characters 
#     text = re.sub(r'([a-zA-Z])\1{2,}\b',r'\1\1',text)
#     text = re.sub(r'([a-zA-Z])\1\1{2,}\B',r'\1\1\1',text)
#     text = re.sub(r'[ ]{2,}',' ',text).strip()   
#     text = re.sub(r'[ ]{2,}',' ',text).strip()   
    
    words=re.split(r'\W+', text)                   ### Split by remaining non-alphanumeric characters 
    non_stop_words=[word for word in words if word not in stopword]        ### Remove stopwords
    lem_words= [wn.lemmatize(word) for word in non_stop_words]             ### Clean text ==
    clean_text= ' '.join(lem_words)
    return clean_text

In [ ]:
comments['clean_comment']=comments['comment_text'].apply(clean)

In [ ]:
comments

In [ ]:
count_vect=CountVectorizer()

In [ ]:
X_counts=count_vect.fit_transform(comments['clean_comment'])

In [ ]:
vocab=count_vect.get_feature_names()

In [ ]:
len(vocab)

In [ ]:
#vocab

In [ ]:
vs_model=pd.DataFrame(X_counts.toarray(), columns=vocab)

In [ ]:
sums=vs_model.apply(sum)

In [ ]:
#np.argsort(sums)
sums

In [ ]:
sums1=sums[sums<10]

In [ ]:
columns_remove=pd.DataFrame(sums1)
columns_remove.reset_index(inplace=True)
columns_remove=columns_remove.rename(columns={'index':'columns'})
columns_remove

In [ ]:
vs_model=vs_model.drop([columns for columns in columns_remove['columns']], axis = 1)
vs_model

In [ ]:
train_data=pd.concat([comments[comments.columns[2:8]], vs_model], axis=1)

In [ ]:
#variety=train_data.apply(sum)

In [ ]:
#variety[0:6]

In [ ]:
tags=comments[comments.columns[2:8]]

In [ ]:
# split into input (X) and output (y) variables
X = np.asarray(vs_model)
y = np.asarray(tags)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.1)

In [ ]:
len(y_test)

In [ ]:
# define the keras model
model = Sequential()
model.add(Dense(5000, input_dim=len(vs_model.columns), activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='sigmoid'))
model.add(Dense(6, activation='sigmoid'))

In [ ]:
# compile the keras model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=3, batch_size=10)

In [ ]:
# evaluate the keras model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

In [ ]:
# make class predictions with the model
predictions = (model.predict(X_test)).astype(int)

In [ ]:
test_data = pd.read_csv("../input/jigsaw-toxic-severity-rating/comments_to_score.csv")
#test_data = pd.read_csv("../input/jigsaw-toxic-comment-classification-challenge/test.csv")
sample_submission = pd.read_csv("../input/jigsaw-toxic-severity-rating/sample_submission.csv")

In [ ]:
test_comments=test_data[0:10000]

In [ ]:
test_comments

In [ ]:
tfidf_trans=TfidfVectorizer()

In [ ]:
X_tfidf=tfidf_trans.fit_transform(comments['clean_comment'])

In [ ]:
X_tfidf.toarray()

In [ ]:
tfidf_test=tfidf_trans.transform(test_comments['text'])

In [ ]:
tfidf_model=pd.DataFrame(tfidf_test.toarray(), columns=vocab)

In [ ]:
tfidf_model

In [ ]:
tfidf_model=tfidf_model.drop([columns for columns in columns_remove['columns']], axis = 1)

In [ ]:
predictions = (model.predict(np.asarray(tfidf_model))).astype(int)

In [ ]:
predictions

In [ ]:
test_prediction=pd.DataFrame(predictions,columns=['toxic','severe_toxic','obscene','threat','insult','identity_hate'])

In [ ]:
test_prediction_all=pd.concat([test_comments, test_prediction],axis=1)

In [ ]:
test_prediction_all[test_prediction_all.threat==1]['text']

In [ ]:
submission = test_prediction_all.drop(['text'],axis=1)

In [ ]:
submission['score']=submission['toxic']*0.166+submission['severe_toxic']*0.166+submission['obscene']*0.166+submission['threat']*0.166+submission['insult']*0.166+submission['identity_hate']*0.166
submission['score']=round(submission['score'], 1)

In [ ]:
sample_submission=submission[['comment_id', 'score']]

In [ ]:
sample_submission.to_csv("submission.csv", index=False)

In [ ]:
submission[['comment_id', 'score']][submission.score>0]